# 5.3 Lab: Optimizar Consultas Práctico

- Objetivo: refactorizar consultas eliminando JOINs innecesarios, usando índices, midiendo tiempos.
- Prerrequisitos: conocimiento de índices y planes de ejecución.
- Entregables: consultas con comentarios de mejora (plan estimado/real).
- Errores comunes: JOIN sin ON, índices no cubiertos, no medir impacto real.

## ¿Por qué optimizar consultas?

Una consulta mal escrita puede tardar segundos o minutos cuando debería ejecutarse en milisegundos. Esto impacta:
- **Experiencia de usuario:** dashboards lentos, reportes que no cargan
- **Costos de infraestructura:** mayor CPU/RAM necesaria
- **Escalabilidad:** sistema que funciona con 1000 registros colapsa con 1 millón

**Para qué:** Reducir tiempos de respuesta, mejorar concurrencia (más usuarios simultáneos), disminuir costos operativos.

**Cómo:** Eliminando JOINs innecesarios, creando índices estratégicos, evitando funciones en WHERE sobre columnas indexadas, usando EXISTS en lugar de IN para grandes volúmenes.

## Técnicas de Optimización

### 1. Eliminar JOINs innecesarios
Si solo necesitas datos de fact_ventas, no hagas JOIN con dim_productos. Usa JOIN únicamente cuando necesites columnas de múltiples tablas.

### 2. Índices cubiertos (covering indexes)
Un índice que contiene todas las columnas del SELECT + WHERE + ORDER BY evita acceso a la tabla base (bookmark lookup).

### 3. Medir con SET STATISTICS
```sql
SET STATISTICS TIME ON;
SET STATISTICS IO ON;
-- Tu consulta aquí
```
Esto muestra lecturas lógicas (páginas leídas de caché) y tiempo de CPU/transcurrido.

### 4. Analizar plan de ejecución
En SSMS: Ctrl+M (plan estimado) o Ctrl+L (plan actual). Busca:
- Table Scan (malo) vs Index Seek (bueno)
- Costos relativos altos (> 50%)
- Warnings (conversiones implícitas, falta de estadísticas)

In [ ]:
-- Conteo de ventas sin JOIN innecesario
SELECT COUNT(*) AS ventas_totales FROM dbo.fact_ventas;

-- Consulta que aprovecha índice compuesto (conceptual)
SELECT producto_id, SUM(cantidad) AS unidades
FROM dbo.fact_ventas
WHERE fecha BETWEEN '2024-01-01' AND '2024-12-31' AND producto_id IN (101,102)
GROUP BY producto_id;

## Ejercicios Prácticos

🟢 **Básico:** Comparar tiempo de ejecución de `SELECT *` vs `SELECT columnas_específicas` en fact_ventas (100k registros)

🟠 **Intermedio:** Refactorizar consulta que hace JOIN con dim_clientes solo para filtrar por segmento='Premium', moviendo filtro a WHERE con subconsulta

🔴 **Avanzado:** Analizar plan de ejecución de agregación con múltiples JOINs, identificar tabla con mayor costo y proponer índice compuesto

⚙️ **Reto Senior:** Crear benchmark comparando 3 enfoques para mismo resultado: subconsulta correlacionada vs CTE vs tabla temporal, documentando lecturas lógicas y tiempo CPU de cada uno

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.